# Project: Identifying Toxic Comments 


The term cyberbullying was probably born together with messaging technologies. With the creation of social networks like Facebook, Myspace, Instagram, and Twitter, cyberbullying has increased even more. Therefore, effective means of coping with it are very important. Luckily, due to the increase in computing power of the newest technology and recent advances in algorithms we have the opportunity to create models to detect bullying messages. Besides, adolescents experiencing social and emotional difficulties were more likely to be cyberbullied and traditionally bullied, than traditionally bullied only (Cross D., 2015). Therefore, after applying the model victims could be offered special tuition on how to defend against bullying, while the bullies could be warned or banned from social networks. In addition, children can be protected from hate speech and profanity.


## Dataset

Since this project is mainly inspired by competition called "Toxic Comment Classification Challenge
" at Kaggle.com, we will use the dataset available [there](https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/data). It contains a large number of Wikipedia comments which have been labeled by human raters for toxic behavior. The types of toxicity are:

- toxic
- severe_toxic
- obscene
- threat
- insult
- identity_hate

We will create a model which predicts a probability of each type of toxicity for each comment.


## Importing Data
**File descriptions**

- train.csv - the training set, contains comments with their binary labels
- test.csv - the test set, you must predict the toxicity probabilities for these comments. To deter hand labeling, the test set contains some comments which are not included in scoring.
- test_labels.csv - labels for the test data; 

Important Note: We will have to drop some of the observations from the test set with labels equal to -1. The reason for it is that the competition provides labels for that part of the data which was used for scoring. Other labels were recorded as -1, we will have to drop them. 

In [1]:
import numpy as np
import pandas as pd

class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

train = pd.read_csv('train.csv').fillna(' ')

# Let's import test data and drop the labels
test_labels = pd.read_csv('test_labels.csv').fillna(' ')
indices_with_scores = test_labels[test_labels['toxic'] != -1].index.values
test_labels = test_labels.iloc[indices_with_scores]

test = pd.read_csv('test.csv').fillna(' ')
test = test.iloc[indices_with_scores]

## Feature Extraction

In order to extract features of data we will estimate TF-IDF metric available in sklearn as TfidfVectorizer.

Term frequency-inverse document frequency (TF-IDF) is a feature vectorization method widely used in text mining to reflect the importance of a term to a document in the corpus. Denote a term by t, a document by d, and the corpus by D. Term frequency TF(t,d) is the number of times that term t appears in document d, while document frequency DF(t,D) is the number of documents that contains term t. If we only use term frequency to measure the importance, it is very easy to over-emphasize terms that appear very often but carry little information about the document, e.g. “a”, “the”, and “of”. If a term appears very often across the corpus, it means it doesn’t carry special information about a particular document. Inverse document frequency is a numerical measure of how much information a term provides:

IDF(t,D)=log( (|D|+1) / (DF(t,D)+1) )

where |D| is the total number of documents in the corpus. Since logarithm is used, if a term appears in all documents, its IDF value becomes 0. Note that a smoothing term is applied to avoid dividing by zero for terms outside the corpus. The TF-IDF measure is simply the product of TF and IDF:
TFIDF(t,d,D)= TF(t,d) ⋅ IDF(t,D)

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

train_text = train['comment_text']
test_text = test['comment_text']

word_vectorizer = TfidfVectorizer(
    strip_accents='unicode',
    analyzer='word',
    stop_words='english',
    ngram_range=(1, 5),
    max_features=100000)
word_vectorizer.fit(train_text)
train_features = word_vectorizer.transform(train_text)
test_features = word_vectorizer.transform(test_text)

## Evaluation Metrics

The Receiver's Operating Characteristic's Area Under the Curve (AUC) is an appropriate metric to evaluate the performance of the model. The score would be the average of the individual AUCs of each predicted column. ROC is a plot of signal (True Positive Rate) against noise (False Positive Rate). The model performance is determined by looking at the area under the ROC curve (or AUC). The best possible AUC is 1 while the worst is 0.5 (the 45 degrees random line). Any value less than 0.5 means we can simply do the exact opposite of what the model recommends to get the value back above 0.5.

## Benchmark model

The winning solution of "Toxic Comment Classification Challenge" at Kaggle.com uses neural networks and sophisticated data preprocessing techniques, achieving ROC AUC score of 0.9885. 

## Model

After testing a few Supervised Machine Learning models we found that Logistic Regression model with 'saga' (optimized for very large datasets) solver provides the best ROC AUC metric. We decided not to use Deep Neural Networks due to much quicker training time and satisfactory performance of Logistic Regression model.

Logistic regression, despite its name, is a linear model for classification rather than regression. Logistic regression is also known in the literature as logit regression, maximum-entropy classification (MaxEnt) or the log-linear classifier. In this model, the probabilities describing the possible outcomes of a single trial are modeled using a logistic function.

The implementation of logistic regression in scikit-learn is accessed from class LogisticRegression. This implementation can fit binary, One-vs- Rest, or multinomial logistic regression with optional L2 or L1 regularization. We will be using binary logistic regression for each label separately with L2 regularization, since it simply performed better. We chose solver 'saga', since it supports very large datasets and exhibits superior performance to solver 'sag'. 

## Implementation

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
scores = []
predictions = pd.DataFrame.from_dict({'id': test['id']})
for class_name in class_names:
    train_target = train[class_name]
    classifier = LogisticRegression(solver='saga')

    cv_score = np.mean(cross_val_score(classifier, train_features, train_target, cv=3, scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name, cv_score))

    classifier.fit(train_features, train_target)
    predictions[class_name] = classifier.predict_proba(test_features)[:, 1]

print('Total CV score is {}'.format(np.mean(scores)))

CV score for class toxic is 0.9682810866794479
CV score for class severe_toxic is 0.9835496418068875
CV score for class obscene is 0.9839129375435997
CV score for class threat is 0.981051826313014
CV score for class insult is 0.9749755180051173
CV score for class identity_hate is 0.9728141593803329
Total CV score is 0.9774308616213999


# Model Validation on unseen test data
Finally, let's calculate the average ROC AUC score for the test set:

In [4]:
from sklearn.metrics import roc_auc_score
final_scores = []
for class_name in class_names:
    test_target = test_labels[class_name]
    score = roc_auc_score(test_labels[class_name], predictions[class_name])
    
    print('ROC AUC score for class {} is {}'.format(class_name, score))
    final_scores.append(score)

print('Average ROC AUC score is {}'.format(np.mean(final_scores)))

ROC AUC score for class toxic is 0.9587563034353864
ROC AUC score for class severe_toxic is 0.9859015352896181
ROC AUC score for class obscene is 0.973170073140212
ROC AUC score for class threat is 0.9851995605743867
ROC AUC score for class insult is 0.966014536374373
ROC AUC score for class identity_hate is 0.9769557449960696
Average ROC AUC score is 0.9743329589683408


# Conclusion

The ROC AUC score is rather high for such a simple model. However, further directions of improvement could be:
- Finding optimal LogisticRegression hyperparameters using GridSearchCV module.
- Adding additional feature set by using TfidfVectorizer on character level.
- Since not all of the data points are in English language, translating all the data points to English.
- Using Deep Neural Networks, preferably employing transfer learning techniques.

## References:
   - Cross, D., Lester, L., & Barnes, A. (2015). A longitudinal study of the social and emotional predictors and consequences of cyber and traditional bullying victimisation. International Journal of Public Health, 60(2), 207-217. doi:10.1007/s00038-015-0655-1
   - (n.d.). Retrieved September 5, 2018, from https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/data
   - Extracting, transforming and selecting features. (n.d.). Retrieved from https://spark.apache.org/docs/2.2.0/ml-features.html#tf-idf
   - Nazrul, S., & Nazrul, S. S. (2018, June 29). Receiver Operating Characteristic Curves Demystified (in Python). Retrieved from https://towardsdatascience.com/receiver-operating-characteristic-curves-demystified-in-python-bd531a4364d0
   - Generalized Linear Models. (n.d.). Retrieved September 4, 2018, from http://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
